Apesar de os dados do GT Educação terem sido tratados nos notebooks anteriores, é necessário realizar algumas transformações adicionais para facilitar a visualização dos dados no Qlik Sense. Vamos ajustar os dados realizando algumas transformações e filtros adicionais.

In [2]:
import pandas as pd
import os

# Carregando os dados

In [3]:
base_path = os.path.join('data_output', 'educacao')

In [4]:
alunos_24 = pd.read_csv(os.path.join(base_path, 'perfil-dos-educandos.csv'),
                        sep=';',
                        decimal=',',
                        encoding='utf-8')

In [5]:
escolas_24 = pd.read_csv(os.path.join(base_path, 'escolas-municipais.csv'),
                        sep=';',
                        decimal=',',
                        encoding='utf-8')

In [ ]:

if not os.path.exists(base_path):
    os.makedirs(base_path)

for name, df in [('demanda e matriculas', mat_24),
                 ('servidores-perfil', perf_24),
                 ('escolas-municipais', escolas_24),
                 ('perfil-dos-educandos', alunos_24),
                 ('educandos-estrangeiros', alunos_est_2024),
                 ('beneficiarios-pbf', alunos_bf_2024),
                 ('nivel-padronizado', nivel_padronizado),
                 ('orcamento', df_orcamento_completo)]:
    
    filepath = os.path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='utf-8')

# Transformando os dados

## Filtrando os dados de escolas

In [7]:
tipoesc_validos = [
    'CCI/CIPS',
    'CECI',
    'CEI DIRET',
    'CEI INDIR',
    'CEMEI',
    'CEU CEI',
    'CEU CEMEI',
    'CEU EMEF',
    'CEU EMEI',
    'CIEJA',
    'CMCT',
    'CR.P.CONV',
    'EMEBS',
    'EMEF',
    'EMEF P FOM',
    'EMEFM',
    'EMEI',
    'EMEI P FOM',
    'MOVA'
]

In [8]:
escolas_24_filtrado = escolas_24[escolas_24['TIPOESC'].isin(tipoesc_validos)].copy()
escolas_24_filtrado

,CODESC,TIPOESC,NOMES,SUBPREF,DISTRITO,CODINEP,CD_CIE,NOME_ANT,REDE,NOMES_COMPLETO,nm_distrito_municipal,cd_subprefeitura,nm_subprefeitura
0,86,EMEI,PAULO CAMILHIER FLORENCANO,GUAIANASES,GUAIANASES,35098711.0,1678110.0,NaN,DIR,EMEI PAULO CAMILHIER FLORENCANO,GUAIANASES,28,GUAIANASES
1,94,EMEI,VICENTE PAULO DA SILVA,CASA VERDE/CACHOEIRINHA,LIMAO,35098361.0,1670158.0,NaN,DIR,EMEI VICENTE PAULO DA SILVA,LIMAO,4,CASA VERDE-CACHOEIRINHA
2,108,EMEF,"JOSE ERMIRIO DE MORAES, SEN.",SAO MIGUEL,JARDIM HELENA,35098760.0,1677116.0,VILA NITRO OPERARIA,DIR,"EMEF JOSE ERMIRIO DE MORAES, SEN.",JARDIM HELENA,23,SAO MIGUEL
3,191,EMEF,"ALIPIO CORREA NETO, PROF.",BUTANTA,VILA SONIA,35098462.0,1679154.0,NaN,DIR,"EMEF ALIPIO CORREA NETO, PROF.",VILA SONIA,10,BUTANTA
4,205,EMEBS,"VERA LUCIA APARECIDA RIBEIRO, PROFA.",PIRITUBA,PIRITUBA,35079029.0,1671138.0,NaN,DIR,"EMEBS VERA LUCIA APARECIDA RIBEIRO, PROFA.",PIRITUBA,2,PIRITUBA-JARAGUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4142,501690,MOVA,INSTITUTO BATISTA BETEL,CIDADE TIRADENTES,CIDADE TIRADENTES,NaN,NaN,NaN,CON,MOVA INSTITUTO BATISTA BETEL,CIDADE TIRADENTES,31,CIDADE TIRADENTES
4143,501691,MOVA,CENTRAL DE ASSOCIACOES E SOCIEDADES POPULARES V,CIDADE TIRADENTES,CIDADE TIRADENTES,NaN,NaN,CENTRAL DE ASSOCIACOES E SOCIEDADES POPULARES,CON,MOVA CENTRAL DE ASSOCIACOES E SOCIEDADES POPUL...,CIDADE TIRADENTES,31,CIDADE TIRADENTES
4144,501694,MOVA,CENTRO DE EDUCACAO DA ZONA LESTE X,PENHA,ARTUR ALVIM,NaN,NaN,NaN,CON,MOVA CENTRO DE EDUCACAO DA ZONA LESTE X,ARTUR ALVIM,21,PENHA
4145,505566,MOVA,ENTIDADE DE PROMOCAO E ASSIST SOCIAL ESPACO AB...,CIDADE ADEMAR,PEDREIRA,NaN,NaN,ENTIDADE DE PROMOCAO E ASS SOCIAL ESPACO ABER,CON,MOVA ENTIDADE DE PROMOCAO E ASSIST SOCIAL ESPA...,PEDREIRA,16,CIDADE ADEMAR


## Ajustando as etapas e níveis de ensino de alunos

In [10]:
alunos_est_2024_desc_modal = {
    'CRECHE': 'Creches',
    'PRE': 'Pré-Escola',
    'FUND': 'Ensino Fundamental',
    'EJA': 'EJA/CIEJA',
    'MOVA': 'MOVA (Parceira)',
    'MEDIO': 'Ensino Médio',
    'EDPROF': 'Educação Profissional',
    'ESPEC': 'EMEBS',
    'CONVEE': 'Educação Especial (Parceira)'
}

alunos_24['DESC_MODAL'] = alunos_24['MODAL'].map(alunos_est_2024_desc_modal)
alunos_24

,DISTRITO,REDE,MODAL,SEXO,NEE,RACA,Qtde,Nível padronizado,nm_distrito_municipal,cd_subprefeitura,nm_subprefeitura,nivel_subprefeitura,DESC_MODAL
0,VILA SONIA,DIR,FUND,F,NaN,PARDA,3,Ensino Fundamental,VILA SONIA,10,BUTANTA,Ensino Fundamental|BUTANTA,Ensino Fundamental
1,VILA SONIA,DIR,FUND,F,NaN,BRANCA,3,Ensino Fundamental,VILA SONIA,10,BUTANTA,Ensino Fundamental|BUTANTA,Ensino Fundamental
2,VILA SONIA,DIR,FUND,F,NaN,PARDA,8,Ensino Fundamental,VILA SONIA,10,BUTANTA,Ensino Fundamental|BUTANTA,Ensino Fundamental
3,VILA SONIA,DIR,FUND,F,NaN,PRETA,2,Ensino Fundamental,VILA SONIA,10,BUTANTA,Ensino Fundamental|BUTANTA,Ensino Fundamental
4,VILA SONIA,DIR,FUND,F,DEFICIENCIA INTELECTUAL,BRANCA,1,Ensino Fundamental,VILA SONIA,10,BUTANTA,Ensino Fundamental|BUTANTA,Ensino Fundamental
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267825,SAO MATEUS,CON,CONVEE,M,DEFICIENCIA INTELECTUAL,PARDA,1,Educação Especial,SAO MATEUS,30,SAO MATEUS,Educação Especial|SAO MATEUS,Educação Especial (Parceira)
267826,SAO MATEUS,CON,CONVEE,M,DEFICIENCIA INTELECTUAL,BRANCA,2,Educação Especial,SAO MATEUS,30,SAO MATEUS,Educação Especial|SAO MATEUS,Educação Especial (Parceira)
267827,SAO MATEUS,CON,CONVEE,M,DEFICIENCIA INTELECTUAL,PARDA,1,Educação Especial,SAO MATEUS,30,SAO MATEUS,Educação Especial|SAO MATEUS,Educação Especial (Parceira)
267828,SAO MATEUS,CON,CONVEE,M,DEFICIENCIA INTELECTUAL,BRANCA,1,Educação Especial,SAO MATEUS,30,SAO MATEUS,Educação Especial|SAO MATEUS,Educação Especial (Parceira)


Vamos conferir se o map foi aplicado corretamente checando a contagem dos valores únicos nas combindações das colunas `MODAL` e `DESC_MODAL`.

In [11]:
alunos_24[['MODAL', 'DESC_MODAL']].value_counts()

MODAL   DESC_MODAL                  
CRECHE  Creches                         102313
FUND    Ensino Fundamental              101168
PRE     Pré-Escola                       36396
EJA     EJA/CIEJA                        15042
MOVA    MOVA (Parceira)                   8215
EDPROF  Educação Profissional             2307
CONVEE  Educação Especial (Parceira)      1141
MEDIO   Ensino Médio                       667
ESPEC   EMEBS                              581
Name: count, dtype: int64

Como temos apenas modalidades com as descrições pretendidas, podemos concluir que o mapeamento foi aplicado corretamente.